In [ ]:
import ast
import contextily as ctx
import psycopg2.extras as pgx
import psycopg2 as pg
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import ast
from bisect import bisect_left


import sqlalchemy
from sqlalchemy import create_engine
import pyarrow as pa
import pyarrow.parquet as pq
from IPython.display import HTML
import base64

from pylab import *
%matplotlib inline
import geopandas as gpd

In [ ]:
# Pull most recent schema names

# Direct psycopg2 connection
conn = pg.connect(
    dbname="dgendb",
    user="postgres",
    password="postgres",
    host="127.0.0.1",
    port=5432
)

query = """
SELECT schema_name
FROM information_schema.schemata
WHERE schema_name LIKE 'diffusion_results_%'
"""

# Use the raw psycopg2 connection
recent_schemas = pd.read_sql(query, conn)

In [ ]:
for schema in recent_schemas.schema_name:
    print(schema)

In [ ]:
# Load data
avg_prices = pd.read_csv("../../../data/average_retail_elec_price.csv")

In [ ]:
baseline_schemas = [
"diffusion_results_baseline_nj_2040_20250804_060534110823",
"diffusion_results_baseline_ar_2040_20250804_060212164035",
"diffusion_results_baseline_id_2040_20250804_052551958214",
"diffusion_results_baseline_ok_2040_20250804_052549768205",
"diffusion_results_baseline_al_2040_20250804_052548614798"
]


policy_schemas = [
"diffusion_results_policy_al_2040_20250804_060448914500",
"diffusion_results_policy_ok_2040_20250804_060444247858",
"diffusion_results_policy_id_2040_20250804_054438482972",
"diffusion_results_policy_ar_2040_20250804_052549955818",
"diffusion_results_policy_nj_2040_20250804_052549688326"
]

# Helper to query and tag results
def load_agent_outputs(schema_name, scenario_label):
    query = f'SELECT * FROM "{schema_name}".agent_outputs'
    df = pd.read_sql(query, conn)  # or psycopg2 conn
    df["scenario"] = scenario_label
    df["schema"] = schema_name
    return df

# Aggregate all results
all_dfs = []

for schema in baseline_schemas:
    print(f"Loading baseline schema: {schema}")
    all_dfs.append(load_agent_outputs(schema, "baseline"))

for schema in policy_schemas:
    print(f"Loading policy schema: {schema}")
    all_dfs.append(load_agent_outputs(schema, "policy"))

# Concatenate all results
agent_outputs_all = pd.concat(all_dfs, ignore_index=True)

In [ ]:
df = pd.read_csv("../../../data/saved_outputs/test_run_five_states_minimize_payback.csv")

In [ ]:
# -- 1. Add total bill savings
df["total_bill_savings"] = df["first_year_elec_bill_savings"] * df["number_of_adopters"]

# -- 2. Get solar technical potential by 2040
tech_2040 = df[df["year"] == 2040].copy()

# -- 2a. Aggregate total adopters and customers
tech_agg = (
    tech_2040.groupby(["state_abbr", "scenario"])[["number_of_adopters", "customers_in_bin"]]
    .sum()
    .reset_index()
)

# -- 2b. Compute percent of technical potential reached
tech_agg["percent_tech_potential"] = (
    tech_agg["number_of_adopters"] / tech_agg["customers_in_bin"]
) * 100

# -- 2. Compute model-based average electricity prices for 2026 (cents/kWh)
model_2026 = df[(df["year"] == 2026) & (df['scenario'] == 'baseline')].copy()

# -- 3. Calculate average prices by state
avg_prices_model = (
    model_2026.groupby("state_abbr")["avg_elec_price_cents_per_kwh"]
    .mean()
    .reset_index()
)

# -- 4. Merge model and EIA prices
price_comparison = pd.merge(avg_prices_model, avg_prices, on="state_abbr", how="inner")
price_comparison['avg_elec_price_cents_per_kwh'] = price_comparison['avg_elec_price_cents_per_kwh']*100

# -- 5. Melt for grouped bar plot
price_melted = price_comparison.melt(
    id_vars="state_abbr",
    value_vars=["avg_elec_price_cents_per_kwh", "cents_per_kwh"],
    var_name="Source",
    value_name="Average Price (¢/kWh)"
)

# Rename for clarity
price_melted["Source"] = price_melted["Source"].map({
    "avg_elec_price_cents_per_kwh": "Model",
    "cents_per_kwh": "EIA"
})

# -- 3. Aggregate metrics

# 3a. Median system size by state/year/scenario
median_kw = (
    df.groupby(["state_abbr", "year", "scenario"])["system_kw"]
    .median()
    .reset_index()
)

# 3c. Total bill savings (already computed in df)
total_bill_savings = (
    df.groupby(["state_abbr", "year", "scenario"])["total_bill_savings"]
    .sum()
    .reset_index()
)

# Market share reached
df['market_potential'] = df['customers_in_bin']*df['max_market_share']
market_share_reached = (
    df.groupby(["state_abbr", "year", "scenario"], as_index=False)
    .agg(market_potential=("market_potential","sum"),
         market_reached=("number_of_adopters", "sum"))
)
market_share_reached['market_share_reached'] = market_share_reached['market_reached']/market_share_reached['market_potential']

# --------------------- PLOTS ---------------------

# 1. Median System Size (kW)
g = sns.FacetGrid(median_kw, col="state_abbr", col_wrap=4, height=3.5, sharey=False)
g.map_dataframe(sns.lineplot, x="year", y="system_kw", hue="scenario", marker="o")
g.set_titles("{col_name}")
g.set_axis_labels("Year", "Median System Size (kW)")
g.set(xticks=[2026, 2030, 2035, 2040])
g.add_legend()
g.fig.suptitle("Median PV System Size by Scenario", y=1.02)
plt.tight_layout()
plt.show()

# -- 1. Get state order based on model-estimated price
model_sorted = (
    price_melted[price_melted["Source"] == "Model"]
    .sort_values("Average Price (¢/kWh)", ascending=False)
    ["state_abbr"]
    .tolist()
)

# -- 2. Plot with sorted x-axis
plt.figure(figsize=(14, 6))
ax = sns.barplot(
    data=price_melted,
    x="state_abbr",
    y="Average Price (¢/kWh)",
    hue="Source",
    order=model_sorted,
    errorbar=None
)

# -- 3. Add white text labels to each bar
for container in ax.containers:
    for bar in container:
        height = bar.get_height()
        if height > 0:
            ax.text(
                bar.get_x() + bar.get_width() / 2,
                height - 1.5,  # shift slightly below the top
                f"{height:.1f}",
                ha='center',
                va='top',
                color='white',
                fontsize=9,
                fontweight='bold'
            )

# -- 4. Style
plt.title("Average Electricity Price in 2026: Model vs EIA (Sorted by Model)")
plt.ylabel("¢/kWh")
plt.xlabel("State")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# -- 1. Get state order based on policy scenario percentage
policy_sorted = (
    tech_agg[tech_agg["scenario"] == "policy"]
    .sort_values("percent_tech_potential", ascending=False)
    ["state_abbr"]
    .tolist()
)

# -- 2. Plot using the sorted state order
plt.figure(figsize=(14, 6))
ax = sns.barplot(
    data=tech_agg,
    x="state_abbr",
    y="percent_tech_potential",
    hue="scenario",
    order=policy_sorted,
    errorbar=None
)

# -- 3. Add white labels
for container in ax.containers:
    for bar in container:
        height = bar.get_height()
        if height > 0:
            ax.text(
                bar.get_x() + bar.get_width() / 2,
                height - 1.5,
                f"{height:.1f}%",
                ha='center',
                va='top',
                color='white',
                fontsize=9,
                fontweight='bold'
            )

# -- 4. Labels and styling
plt.title("Solar Technical Potential Reached in 2040 (Sorted by Policy %)")
plt.ylabel("Percent of Technical Potential (%)")
plt.xlabel("State")
plt.xticks(rotation=45)
plt.ylim(0, tech_agg["percent_tech_potential"].max() * 1.1)
plt.tight_layout()
plt.show()

# 4. Total Bill Savings by Scenario
g = sns.FacetGrid(total_bill_savings, col="state_abbr", col_wrap=4, height=3.5, sharey=False)
g.map_dataframe(sns.lineplot, x="year", y="total_bill_savings", hue="scenario", marker="o")
g.set_titles("{col_name}")
g.set_axis_labels("Year", "Total First-Year Bill Savings ($)")
g.set(xticks=[2026, 2030, 2035, 2040])
g.add_legend()
g.fig.suptitle("Total First-Year Bill Savings by Scenario", y=1.02)
plt.tight_layout()
plt.show()


metrics = {
    "number_of_adopters": "Number of Adopters",
    "system_kw_cum": "Cumulative Deployment (kW)",
    "batt_kwh_cum": "Cumulative Battery Size (kWh)"
}

# -- 3. Loop over metrics and plot by state_abbr
for col, label in metrics.items():
    # Aggregate data
    grouped = df.groupby(["state_abbr", "year", "scenario"])[col].sum().reset_index()

    # Faceted line plot
    g = sns.FacetGrid(grouped, col="state_abbr", col_wrap=4, height=3.5, sharey=False)
    g.map_dataframe(sns.lineplot, x="year", y=col, hue="scenario", marker="o")
    g.set_titles(col_template="{col_name}")
    g.set_axis_labels("Year", label)
    g.set(xticks=[2026, 2030, 2035, 2040])
    g.add_legend()
    g.fig.suptitle(f"{label} by Scenario (Baseline vs Policy)", y=1.02)
    plt.tight_layout()
    plt.show()


In [ ]:
df['prop_system_size'] = df['system_kw']/df['max_system_kw']
df[(df['year'] == 2026) & (df['scenario'] == "policy")].groupby(['state_abbr'], as_index=False).agg({'initial_number_of_adopters':'sum'})

In [ ]:
df_avg_price = (
    df.groupby(['state_abbr', 'year', 'scenario'])['avg_elec_price_cents_per_kwh']
    .mean()
    .reset_index()
)

# Set up the FacetGrid
g = sns.FacetGrid(df_avg_price, col="state_abbr", col_wrap=5, height=3.5, sharey=False)
g.map_dataframe(sns.lineplot, x="year", y="avg_elec_price_cents_per_kwh", hue="scenario", marker="o")

# Format plot
g.set_titles("{col_name}")
g.set_axis_labels("Year", "Avg. Elec Price (¢/kWh)")
g.add_legend(title="Scenario")
g.fig.suptitle("Average Electricity Price by State, Year, and Scenario", y=1.03)

plt.tight_layout()
plt.show()

In [ ]:
df['market_reached'] = df['market_share'] / df['max_market_share']
df['market_reached'] = df['market_reached'].replace([np.inf, -np.inf], np.nan).fillna(0)
plot_df = df[(df['year'] == 2040) & 
             (df['max_market_share'] > 0) & 
             (df['scenario'] == "policy") &
             (df['max_market_share']>=df['market_share'])]

# Create FacetGrid histogram by state
g = sns.FacetGrid(plot_df, col="state_abbr", col_wrap=5, height=3.5, sharey=True)
g.map_dataframe(sns.histplot, x="market_reached", bins=20, kde=False, multiple="layer", alpha=0.6)
g.set_titles("{col_name}")
g.add_legend(title="Scenario")
g.set_axis_labels("Market Reached", "Count")
g.fig.suptitle("Market Reached Distribution by State (2026)", y=1.05)
plt.tight_layout()
plt.show()

In [ ]:
df.groupby(['state_abbr', 'scenario','compensation_style']).agg({'bldg_id':'count'})

In [ ]:
df[df['scenario'] == 'baseline'][['system_kw', 'npv', 'payback_period']].corr()

In [ ]:
# 4. Total Bill Savings by Scenario
g = sns.FacetGrid(df[df['year']==2026], col="state_abbr", col_wrap=5, height=3.5, sharey=False)
g.map_dataframe(sns.histplot, x="prop_system_size", bins=10, kde=False)

g.set_titles("{col_name}")
g.set_axis_labels("System Size / Max System Size", "Count")
g.fig.suptitle(f"Distribution of Sizing Ratio (prop_system_size) by State", y=1.02)


In [ ]:
ri = df[df['state_abbr'] == 'RI']

(
    ri[ri['year'].isin([2033,2034,2035,2036])].groupby(['scenario', 'year'], as_index=False)
    .agg({
        'first_year_elec_bill_with_system':'median',
        'first_year_elec_bill_without_system':'median',
        'first_year_elec_bill_savings':'median', 
        'annual_energy_production_kwh':'median',
        'wholesale_elec_price_dollars_per_kwh':'median',
          'number_of_adopters':'median', 
          'avg_elec_price_cents_per_kwh':'median',
          'new_system_kw':'median',
          'system_kw':'median',
          'max_system_kw':'median',
          'load_kwh_per_customer_in_bin':'median',
          'npv':'median',
          'payback_period':'median'
          })
)

In [ ]:
# Filter to policy scenario
df_policy = df[df["scenario"] == "policy"].copy()

# Group by state and year
grouped = df_policy.groupby(["state_abbr", "year"])[["max_system_kw"]].mean().reset_index()

# Melt for line plot
melted = grouped.melt(
    id_vars=["state_abbr", "year"],
    value_vars=["max_system_kw"],
    var_name="Type",
    value_name="Count"
)

# Plot: faceted line plots by state
g = sns.FacetGrid(melted, col="state_abbr", col_wrap=5, height=3.5, sharey=False)
g.map_dataframe(sns.lineplot, x="year", y="Count", hue="Type", marker="o")
g.set_axis_labels("Year", "Count")
g.set_titles("{col_name}")
g.add_legend(title="")

# Format x-axis ticks
for ax in g.axes.flatten():
    ax.set_xticks([2026, 2030, 2035, 2040])
    ax.set_xticklabels([2026, 2030, 2035, 2040])

plt.subplots_adjust(top=0.9)
g.fig.suptitle("Number of Adopters vs Customers in Bin by Year (Policy Scenario)", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# First build an interpolated set of emissions factors from NREL Cambium

# Parameters
start_year, end_year = 2026, 2050

# Build the LRMER lookup from cambium

#  Pivot so each (state,month,hour) is a row and t-years are columns
years_full = list(range(2025, 2051))
camb_pivot = (
    cambium
    .pivot_table(index=['state','month','hour'],
                 columns='t',
                 values='lrmer_co2e')
    .reindex(columns=years_full)       # ensure all years present
)
# Linearly interpolate along the year-axis
camb_interp = camb_pivot.interpolate(axis=1, limit_area='inside')

# Melt back to long format
camb_long = (
    camb_interp
    .reset_index()
    .melt(id_vars=['state','month','hour'],
          var_name='year',
          value_name='lrmer_co2e')
    .query('year >= @start_year and year <= @end_year')
)
# Final lookup keyed by (state,year,month,hour)
lrmer_lookup = camb_long.set_index(
    ['state','year','month','hour']
)['lrmer_co2e'].to_dict()

In [ ]:
# Assume baseline and policy DataFrames are already loaded:
scenarios = {'Baseline': baseline, 'Policy': policy}

# 1) Cumulative installations, bill savings, and deployment in MW
fig, axes = plt.subplots(3, 1, figsize=(8, 12), sharex=True)

for name, df in scenarios.items():
    # Apply weights and scalars where appropriate
    df['first_year_elec_bill_savings'] = df['first_year_elec_bill_savings'] * df['new_adopters']

    # Aggregate annual metrics
    annual = (
        df.groupby('year')
          .agg(
              installations=('new_adopters', 'sum'),
              bill_savings=('first_year_elec_bill_savings', 'sum'),
              deployment_kw=('system_kw_cum', 'sum')
          )
    )
    # Compute cumulative series
    annual['cum_installations'] = annual['installations'].cumsum()
    annual['cum_bill_savings'] = annual['bill_savings'].cumsum()
    annual['cum_deployment_mw'] = annual['deployment_kw'] / 1000.0

    # Plot
    axes[0].plot(annual.index, annual['cum_installations'], label=name)
    axes[1].plot(annual.index, annual['cum_bill_savings'], label=name)
    axes[2].plot(annual.index, annual['cum_deployment_mw'], label=name)

axes[0].set_ylabel('Cumulative Installations')
axes[1].set_ylabel('Cumulative Bill Savings ($)')
axes[2].set_ylabel('Cumulative Deployment (MW)')
axes[2].set_xlabel('Year')
for ax in axes:
    ax.grid(True)
    ax.legend()
plt.tight_layout()
plt.show()

# 2) Number of positive NPV households annually
plt.figure(figsize=(8, 4))
for name, df in scenarios.items():
    pos_counts = df[df['npv'] > 0].groupby('year')['bldg_id'].count()
    plt.plot(pos_counts.index, pos_counts.values, marker='o', label=name)
plt.ylabel('Positive NPV Agents')
plt.xlabel('Year')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# 3) Median system payback annually
plt.figure(figsize=(8, 4))
for name, df in scenarios.items():
    median_payback = df.groupby('year')['payback_period'].median()
    plt.plot(median_payback.index, median_payback.values, marker='o', label=name)
plt.ylabel('Median Payback Period (yrs)')
plt.xlabel('Year')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# 4) Cumulative GHG emissions savings
# Assumes lrmer_lookup is defined as dict[(state, year, month, hour)] -> kg/MWh

def compute_cumulative_avoided(df, lrmer_lookup):
    cum = []
    for col in ['consumption_hourly_list',
            'generation_hourly_list',
            'batt_dispatch_profile_list']:
        df[col] = df[col].apply(ast.literal_eval)
    for year, group in df.groupby('year'):
        # prepare datetime index for this year
        dtidx = pd.date_range(f'{year}-01-01', periods=8760, freq='h')
        months = dtidx.month
        hours = dtidx.hour
        total_avoided = 0.0
        for _, row in group.iterrows():
            w = row['new_adopters']
            gen = np.array(row['generation_hourly_list'], dtype=float) * row['system_kw'] * w

            # grab batt profile, but if it's empty, replace with zeros
            raw_batt = row['batt_dispatch_profile_list']
            batt_arr = np.array(raw_batt, dtype=float)
            if batt_arr.size != gen.size:
                batt_arr = np.zeros_like(gen)
            batt = batt_arr * w

            # now it's safe to subtract
            avoided = gen - batt
            # look up emissions factors (kg/MWh -> kg/kWh)
            fac = np.array([lrmer_lookup[(row['state_abbr'], year, m, h)] for m, h in zip(months, hours)], dtype=float) / 1000.0
            total_avoided += (avoided * fac).sum() / 1000.0  # convert kg -> metric tons
        cum.append({'year': year, 'avoided_tons': total_avoided})

    ann = pd.DataFrame(cum).set_index('year')
    ann['cum_avoided_tons'] = ann['avoided_tons'].cumsum()
    return ann['cum_avoided_tons']

plt.figure(figsize=(8, 4))
for name, df in scenarios.items():
    cumulative = compute_cumulative_avoided(df, lrmer_lookup)
    plt.plot(cumulative.index, cumulative.values, marker='o', label=name)
plt.ylabel('Cumulative CO₂e Avoided (tons)')
plt.xlabel('Year')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
